**In this script the following things can be done**

1. tranforming DICOM to Niftis
2. generating data structre in accordance with the BIDS standard
    + incl. necessary .json files
    + incl .tsv files with trial-onsets
3. generate .tsv from nrec's .mat files
    + split by trials
    + split by into different trial phases

**init**

In [2]:
import os
from pathlib import Path
import sys
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from nipype.interfaces.dcm2nii import Dcm2niix
from scipy.io import loadmat 
import json
import warnings
from os.path import getsize, exists
from os.path import join as opj
from os import listdir
import glob
import warnings
import importlib
import json
from sklearn.utils import Bunch
from nilearn import plotting
from nilearn import image

# sys.path.append( sys.path.append('/home/marius/ownCloud/PhD/projects/scientific/gaze-motion/analysis/') )

# sys.path.append('/home/marius/ownCloud/PhD/projects/scientific/gaze-motion/analysis/python')
# sys.path.append('/home/marius/ownCloud/PhD/projects/scientific/gaze-motion/analysis/')

### in info the order of runs during data collection is specified which is needed for renaming the dicom files
import fMRIus
# from fMRIus.src.info import info as fmrius_info

    
# importlib.reload(info)



In [4]:
# init new dict if not existing
if 'subjects' not in locals():
    subjects = {}

sub_ses_list = {'sub-00': ['ses-gc', 'ses-com'], 'sub-0000':['ses-com', 'ses-sep']}

for sub_id in sub_ses_list:
    subjects[sub_id] = {}
    if not sub_ses_list[sub_id] == '':
        for ses_ in sub_ses_list[sub_id]:
            subjects[sub_id][ses_] = gma.subject.Subject(sub_id, ses_, gma.info.data_paths)
    else:
        subjects[sub_id] = gma.subject.Subject(sub_id, sub_ses_list[sub_id], gma.info.data_paths)


In [4]:
subject_data['anat']

{'mask': '/media/marius/data_ex/gaze-motion/derivatives/fmriprep/sub-0000/anat/sub-0000_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz',
 'img': '/media/marius/data_ex/gaze-motion/derivatives/fmriprep/sub-0000/anat/sub-0000_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz',
 'left_surf_inflated': '/media/marius/data_ex/gaze-motion/derivatives/fmriprep/sub-0000/anat/sub-0000_hemi-L_inflated.surf.gii',
 'left_surf_pial': '/media/marius/data_ex/gaze-motion/derivatives/fmriprep/sub-0000/anat/sub-0000_hemi-L_pial.surf.gii',
 'right_surf_inflated': '/media/marius/data_ex/gaze-motion/derivatives/fmriprep/sub-0000/anat/sub-0000_hemi-R_inflated.surf.gii',
 'right_surf_pial': '/media/marius/data_ex/gaze-motion/derivatives/fmriprep/sub-0000/anat/sub-0000_hemi-R_pial.surf.gii'}

# Move and convert dicom files

## from downloaded folder structure

In [16]:
sub_id = 'sub-01'
ses_id = 'ses-com'

dicom_dir = opj('/media/marius/data_ex/gaze-motion/dsgm/', sub_id, ses_id, 'dicom/*/*')
root_dir = [el for el in glob.glob(dicom_dir) if '_MR_' in el][0]
root_dir

'/media/marius/data_ex/gaze-motion/dsgm/sub-01/ses-com/dicom/IR01_b1980.01.01/u2023.04.25_MR_Prisma_1'

In [19]:
dicom_dir

'36_cmrr_mbep2d_bold_H20_44_MB2'

In [21]:
### from downloaded folder structure

sub_id = 'sub-02'
ses_id = 'ses-com'

dicom_dir = opj('/media/marius/data_ex/gaze-motion/dsgm/', sub_id, ses_id, 'dicom/*/*')
root_dir = [el for el in glob.glob(dicom_dir) if '_MR_' in el][0]

dest_dir = opj('/media/marius/data_ex/gaze-motion/dsgm/', sub_id, ses_id, 'niftis')

# convert all files in root_dir into to nifti and copy to dest_dir
for dicom_dir in listdir(root_dir):
    
    ### excluded or broken/incomplete runs go into archive
    if not dicom_dir == 'archive':
        c_path = opj( root_dir, dicom_dir )
        converter = Dcm2niix()
        converter.inputs.source_dir = c_path
        converter.inputs.compress = 'n'
        #converter.inputs.compression = 3
        converter.inputs.output_dir = c_path
        converter.cmdline
        'dcm2niix -b y -z y -3 -x n -t n -m n'
        converter.run()#
        if not exists( opj(dest_dir, dicom_dir) ):
            !mkdir {dest_dir}/{dicom_dir}
        !mv {c_path}/*.json {dest_dir}/{dicom_dir}/
        !mv {c_path}/*.nii {dest_dir}/{dicom_dir}/

230426-14:09:10,415 nipype.interface INFO:
	 stdout 2023-04-26T14:09:10.415437:Chris Rorden's dcm2niiX version v1.0.20210317  (JP2:OpenJPEG) (JP-LS:CharLS) GCC5.5.0 x86-64 (64-bit Linux)
230426-14:09:10,417 nipype.interface INFO:
	 stdout 2023-04-26T14:09:10.415437:Found 1 DICOM file(s)
230426-14:09:10,419 nipype.interface INFO:
	 stdout 2023-04-26T14:09:10.415437:Warning: Adjusting for negative MosaicRefAcqTimes (issue 271).
230426-14:09:10,421 nipype.interface INFO:
	 stdout 2023-04-26T14:09:10.415437:Convert 1 DICOM as /media/marius/data_ex/gaze-motion/dsgm/sub-02/ses-com/dicom/JM01_b1980.01.01/u2023.04.25_MR_Prisma_1/17_cmrr_mbep2d_bold_H20_44_MB2_SBRef/17_cmrr_mbep2d_bold_H20_44_MB2_SBRef_cmrr_mbep2d_bold_H20_44_MB2_20230425155555_17 (64x64x44x1)
230426-14:09:10,433 nipype.interface INFO:
	 stdout 2023-04-26T14:09:10.433856:Conversion required 0.572132 seconds (0.029868 for core code).
230426-14:09:13,25 nipype.interface INFO:
	 stdout 2023-04-26T14:09:13.025205:Chris Rorden's dcm

## from DVD-source

In [3]:
### from DVD-source

sub_id = 'sub-01'

source_dir = '/media/marius/DVD/DICOM/20210420/15490000"
dicom_dir = '/media/marius/data_ex/gaze-motion/dsgm/'+sub_id+'/dicom'
output_dir = '/media/marius/data_ex/dsgm/sub-00/nii_from_nipype'
dicom_files = glob.glob(source_dir+'/*/*')

### move dicom files from DVD-source_folder to destination
### dcm2nii requires single root folder
for dcm_file in dicom_files:
    !cp {dcm_file} {dicom_dir}/
    
converter = Dcm2niix()
converter.inputs.source_dir = dicom_dir
converter.inputs.compress = 'n'
#converter.inputs.compression = 3
converter.inputs.output_dir = output_dir
converter.cmdline
'dcm2niix -b y -z y -3 -x n -t n -m n'
converter.run()#

4136

# Move and rename nifit files to BIDS folders: /func/, /anat/, /fmap/

## from downloaded source structure

In [22]:
### FROM DOWNLOADED FOLDER STRUCTURE
########################################

subjects = ['sub-01']
sessions = ['ses-com']

for sub_id in subjects:
    for ses_ in sessions:

        root_dir =  opj('/media/marius/data_ex/gaze-motion/dsgm', sub_id, ses_, 'niftis')
        bids_func_dir = opj('/media/marius/data_ex/gaze-motion/dsgm', sub_id, ses_, 'func')
        bids_anat_dir = opj('/media/marius/data_ex/gaze-motion/dsgm', sub_id, ses_, 'anat')
        bids_fmap_dir = opj('/media/marius/data_ex/gaze-motion/dsgm', sub_id, ses_, 'fmap')

        if not os.path.isdir(bids_func_dir):
            os.mkdir(bids_func_dir)
        if not os.path.isdir(bids_anat_dir):
            os.mkdir(bids_anat_dir)
        if not os.path.isdir(bids_fmap_dir):
            os.mkdir(bids_fmap_dir)

        func_protocol_name = 'cmrr_mbep2d_bold_H20_44_MB2'
        anat_protocol_name = 't1_mprage_sag_fast_grappa2'
        fmap_protocol_name = 'gre_field_mapping'

        fmap_names = ['magnitude1', 'magnitude2', 'phasediff']

        func_source_dirs = list(np.sort(glob.glob( root_dir+'/*'+func_protocol_name  )))
        sbref_source_dirs = list(np.sort(glob.glob( root_dir+'/*'+func_protocol_name+'_SBRef'  )))
        anat_source_dir = glob.glob( root_dir+'/*'+anat_protocol_name  )[0]
        fmap_source_dir = list(np.sort(glob.glob( root_dir+'/*'+fmap_protocol_name  )))


        ### copy functional files
        for iif, func_dir in enumerate(func_source_dirs):
            # !cp {func_dir}/*.json {bids_func_dir}/{sub_id}_task-{func_runs[sub_id][iif]}_bold.json
            # !cp {func_dir}/*.nii {bids_func_dir}/{sub_id}_task-{func_runs[sub_id][iif]}_bold.nii   
            !cp {func_dir}/*.json {bids_func_dir}/{sub_id}_{ses_}_task-{info.func_run_dict[sub_id][ses_]['experimental_order'][iif]}_bold.json
            !cp {func_dir}/*.nii {bids_func_dir}/{sub_id}_{ses_}_task-{info.func_run_dict[sub_id][ses_]['experimental_order'][iif]}_bold.nii  

            ### check of SBRef exists
            if sbref_source_dirs: ### check if there are any SBRef 
                if int(func_dir.split('/')[-1][0:2])-1 == int(sbref_source_dirs[0].split('/')[-1][0:2]):
                    !cp {sbref_source_dirs[0]}/*json {bids_func_dir}/{sub_id}_{ses_}_task-{info.func_run_dict[sub_id][ses_]['experimental_order'][iif]}_sbref.json
                    !cp {sbref_source_dirs[0]}/*nii {bids_func_dir}/{sub_id}_{ses_}_task-{info.func_run_dict[sub_id][ses_]['experimental_order'][iif]}_sbref.nii
                    sbref_source_dirs.pop(0) ### remove SBRef just copied

            ### add 'TaskName' field to json file
            json_file = opj( bids_func_dir, sub_id+'_'+ses_+'_task-'+info.func_run_dict[sub_id][ses_]['experimental_order'][iif]+'_bold.json' )
            with open(json_file, 'r+') as f:
                data = json.load(f)
                data['TaskName'] = info.func_run_dict[sub_id][ses_]['experimental_order'][iif].split('_')[0]
                f.seek(0)        # <--- should reset file position to the beginning.
                json.dump(data, f, indent=4)
                f.truncate()     # remove remaining part   

        ### copy anat
        !cp {anat_source_dir}/*.json {bids_anat_dir}/{sub_id}_{ses_}_T1w.json
        !cp {anat_source_dir}/*.nii {bids_anat_dir}/{sub_id}_{ses_}_T1w.nii

        ### copy fmap
        for iif, fmap_dir in enumerate(fmap_source_dir):
            fmap_files = listdir(fmap_dir)
            for fmf in fmap_files:
                if 'ph' in str(fmf):
                    !cp {fmap_dir}/*ph.nii {bids_fmap_dir}/{sub_id}_{ses_}_phasediff.nii
                    !cp {fmap_dir}/*ph.json {bids_fmap_dir}/{sub_id}_{ses_}_phasediff.json
                    with open(opj(bids_fmap_dir, sub_id+'_'+ses_+'_phasediff.json'), 'r+') as f:
                        data = json.load(f)
                        data['IntendedFor'] = list(np.sort([ses_+'/func/'+el for el in os.listdir( bids_func_dir ) if 'json' not in el]))
                        f.seek(0)        # <--- should reset file position to the beginning.
                        json.dump(data, f, indent=4)
                        f.truncate()     # remove remaining part 
                        
                elif 'e1' in str(fmf):
                    !cp {fmap_dir}/*e1.nii {bids_fmap_dir}/{sub_id}_{ses_}_magnitude1.nii
                    !cp {fmap_dir}/*e1.json {bids_fmap_dir}/{sub_id}_{ses_}_magnitude1.json
                    with open(opj(bids_fmap_dir, sub_id+'_'+ses_+'_magnitude1.json'), 'r+') as f:
                        data = json.load(f)
                        data['IntendedFor'] = list(np.sort([ses_+'/func/'+el for el in os.listdir( bids_func_dir ) if 'json' not in el]))
                        f.seek(0)        # <--- should reset file position to the beginning.
                        json.dump(data, f, indent=4)
                        f.truncate()     # remove remaining part 
                    
                elif 'e2' in str(fmf):
                    !cp {fmap_dir}/*e2.nii {bids_fmap_dir}/{sub_id}_{ses_}_magnitude2.nii
                    !cp {fmap_dir}/*e2.json {bids_fmap_dir}/{sub_id}_{ses_}_magnitude2.json
                    with open(opj(bids_fmap_dir, sub_id+'_'+ses_+'_magnitude2.json'), 'r+') as f:
                        data = json.load(f)
                        data['IntendedFor'] = list(np.sort([ses_+'/func/'+el for el in os.listdir( bids_func_dir ) if 'json' not in el]))
                        f.seek(0)        # <--- should reset file position to the beginning.
                        json.dump(data, f, indent=4)
                        f.truncate()     # remove remaining part 



zsh:1: no matches found: {func_dir}/*.json
zsh:1: no matches found: {func_dir}/*.nii


NameError: name 'info' is not defined

## from DVD source structure


In [121]:
### from DVD-source (without folder structure)

func_protocol_name = 'cmrr_mbep2d_bold_H20_44_MB2'
anat_protocol_name = 't1_mprage_sag_fast_grappa2'
fmap_protocol_name = 'gre_field_mapping'

fmap_names = ['magnitude1', 'magnitude2', 'phasediff']

anat_nii = glob.glob('/media/marius/data_ex/dsgm/sub-00/niftis/*'+anat_protocol_name+'*.nii')[0]
func_files = np.sort(glob.glob('/media/marius/data_ex/dsgm/sub-00/niftis/*'+func_protocol_name+'*'))
run_files = [ff for ff in func_files if ((getsize(ff) > 30000000) and (getsize(ff) < 40000000)) or ((getsize(ff) > 165000000) and (getsize(ff) < 175000000))]

fmap_files = glob.glob('/media/marius/data_ex/dsgm/sub-00/niftis/*'+fmap_protocol_name+'*.nii')

In [125]:
subj = 'sub-00'

### copy functional niftis
cntr = 0
for ii,kk in enumerate(parms.subject_data_fmriprep[subj].keys()):
    if kk[:-1] == 'MTloc_run-' or kk[:2] == 'ec':
        ff = run_files[cntr]
        ### copy func niis
        !cp {ff} {parms.root_dir}/{subj}/func/{subj}_task-{kk}_bold.nii
        ### copy func jsons
        !cp {ff[:-4]}.json {parms.root_dir}/{subj}/func/{subj}_task-{kk}_bold.json
        print(kk[:-6])
        ### add task name to .json
        with open(parms.root_dir+subj+'/func/'+subj+'_task-'+kk+'_bold.json', 'r+') as f:
            data = json.load(f)
            data['TaskName'] = kk[:-6]
            f.seek(0)        # <--- should reset file position to the beginning.
            json.dump(data, f, indent=4)
            f.truncate()     # remove remaining part   
        cntr += 1

### copy anat nifti
!cp {anat_nii} {parms.root_dir}/{subj}/anat/{subj}_T1w.nii
### copy anat json
!cp {anat_nii[:-4]}.json {parms.root_dir}/{subj}/anat/{subj}_T1w.json

### copy fmap niftis
for ff in fmap_files:
    if ff[-6:] == 'ph.nii':
        !cp {ff} {parms.root_dir}/{subj}/fmap/{subj}_phasediff.nii
        !cp {ff[:-4]}.json {parms.root_dir}/{subj}/fmap/{subj}_phasediff.json
    if ff[-6:] == 'e1.nii':
        !cp {ff} {parms.root_dir}/{subj}/fmap/{subj}_magnitude1.nii
        !cp {ff[:-4]}.json {parms.root_dir}/{subj}/fmap/{subj}_magnitude1.json
    if ff[-6:] == 'e2.nii': 
        !cp {ff} {parms.root_dir}/{subj}/fmap/{subj}_magnitude2.nii
#         !cp {ff[:-4]}.json {parms.root_dir}/{subj}/fmap/{subj}_magnitude2.json


MTloc
MTloc
MTloc
MTloc
MTloc
MTloc
MTloc
ec
ec
ec
ec
ec
ec


# Make event .tsv files from nrec files

In [11]:
info.func_run_dict[sub_id][ses_]

{'MTloc': ['MTloc_run-1',
  'MTloc_run-2',
  'MTloc_run-3',
  'MTloc_run-4',
  'MTloc_run-5',
  'MTloc_run-6',
  'MTloc_run-7'],
 'gc': ['gc_run-1',
  'gc_run-2',
  'gc_run-3',
  'gc_run-4',
  'gc_run-5',
  'gc_run-6'],
 'experimental_order': ['MTloc_run-1',
  'MTloc_run-2',
  'MTloc_run-3',
  'MTloc_run-4',
  'MTloc_run-5',
  'MTloc_run-6',
  'MTloc_run-7',
  'gc_run-1',
  'gc_run-2',
  'gc_run-3',
  'gc_run-4',
  'gc_run-5',
  'gc_run-6']}

In [23]:
subjects = ['sub-00']
sessions = ['ses-gc']



onset_alignment = 'cue'     # 'cue' or 'instruction'
onset_offset = 0 # 1.5=instruction end
print(onset_alignment)
tasks = ['gc'] #['MTloc', 'gic', 'co', 'go', 'faceLoc']### only generate event tsv for main experiment, 
iris_color_tTD_mapping = {'bl':'1', 'gr':'2', 'br':'3'} # needed for Iris-color mapping condition
event_duration = 0
 

evd = [] ### array to collect event dicts, if you want


#
for sub_id in subjects:
    
    print('*** Processing subject: ', sub_id)

    for ses_ in sessions:
    
        #
        # get session sequence of cube arrangement
        if sub_id not in ['sub-00']:
            trial_list_dir = os.listdir(opj(info.data_paths.bids_dir, sub_id, ses_, 'nrec/trial_lists'))
            #
            # get nrec mat files
            nrec_dir = np.sort(os.listdir(opj( info.data_paths.bids_dir, sub_id, ses_, 'nrec' )))
            mat_files_of_runs = [ff for ff in nrec_dir if '.mat' in ff and not 'calibration' in ff]
        else:
            None
            # run_identifier = list(pd.read_csv(opj(info.data_paths.bids_dir, sub_id, ses_, 'nrec/trial_lists', 'run_identifier.tsv') , sep='\t', header=None)[0])
            # session_sequence = list(pd.read_csv( opj( parms.data_paths.root_dir, sub_id, 'nrec/trial_lists', 'cube_allignment_sequence.tsv' ) , sep='\t', header=None)[0])
            # cube_MTloc_runs = list(pd.read_csv( opj( parms.data_paths.root_dir, sub_id, 'nrec/trial_lists', 'MTloc_cubes.tsv' ) , sep='\t', header=None)[0])

        

        #
        # sort .mat files according to temporal run sequence
        sorted_ids = np.argsort([ff_t[-22:] for ff_t in mat_files_of_runs])
        mat_files_of_runs = np.array(mat_files_of_runs)[sorted_ids]


        #
        # iterate runs
        for iis, run_ in enumerate(info.func_run_dict[sub_id][ses_]['experimental_order']):

            if run_.split('_')[0] in tasks:
                        

                #
                # init variables
                onset = []
                duration = []
                trial_type = []

                #
                # MTloc
                if 'MTloc' in run_:
                    print(run_)
                    
                    mat = loadmat( opj( info.data_paths.bids_dir, sub_id, ses_, 'nrec', mat_files_of_runs[iis]) )
                    
                    trial_onset = mat['TrialList'][:,8].tolist()
                    offsets = mat['TrialList'][:,9].tolist()
                    tt_diff = np.subtract(trial_onset[1:], offsets[:-1])
                    mean_tt_diff = np.mean(tt_diff[np.intersect1d(np.where(tt_diff > 0.05), np.where(tt_diff < 0.15))])
                    # repair missing trials
                    idxs = np.add(np.where(np.array(trial_onset[1:]) == 0.0)[0], 1)
                    for ii, idx_ in enumerate(idxs):
                        trial_onset[ii+idx_] = offsets[ii-1+idx_] + mean_tt_diff
                        offsets[ii+idx_] = trial_onset[ii+idx_]+30
                        # trial_onset = [0, 30, 60, 90, 120, 150]
                    
                    # sub-00 and sub-000 had special versions of the motion localizers
                    if sub_id in ['sub-00', 'sub-000']:
                        if run_identifier[iis] == 'MTloc_Cubes':
                            for ii, el in enumerate( trial_onset ):
                                onset.append( el )
                                duration.append( 15 )
                                if ii % 2 == 0:
                                    trial_type.append( 'Motion' )
                                else:
                                    trial_type.append( 'Static' )

                        if run_identifier[iis] in ['MTloc_Expanding', 'MTloc_Rotation', 'MTloc_Random']:
                            for ii, el in enumerate( trial_onset ):
                                onset.append( el )
                                onset.append( el+15 )
                                duration.append( 15 )
                                duration.append( 15 )
                                trial_type.append( 'Motion' )
                                trial_type.append( 'Static' )

                    else:
                        for ii, el in enumerate( trial_onset ):
                            onset.append( el )
                            onset.append( el+15 )
                            duration.append( 15 )
                            duration.append( 15 )
                            trial_type.append( 'Motion' )
                            trial_type.append( 'Static' )


                #
                # faceLoc
                elif 'faceLoc' in run_:
                    print(run_)
                    
                    trl_list = pd.read_csv( opj( info.data_paths.bids_dir, sub_id, ses_, 'nrec/trial_lists', run_+'.txt') , sep='\t', header=None)
                    mat = loadmat( opj( info.data_paths.bids_dir, sub_id, ses_, 'nrec', mat_files_of_runs[iis] ) )
                    onset = mat['TrialList'][:,8].tolist()
                    offset = mat['TrialList'][:,9].tolist()
                    
                    # repair missing trial times
                    trial_durations = np.subtract(offset, onset)
                    mean_trial_duration = np.mean(trial_durations[np.intersect1d(np.where(trial_durations > 0.5), np.where(trial_durations < 1))])
                    tt_diff = np.abs(np.subtract(onset[1:], offset[:-1]))
                    mean_tt_diff = np.mean(tt_diff[np.intersect1d(np.where(tt_diff > 0.05), np.where(tt_diff < 0.3))])
                    idxs = np.add(np.where(np.array(onset[1:]) == 0.0)[0], 1)
                    for ii, idx_ in enumerate(idxs):
                        onset[idx_] = offset[idx_-1] + mean_tt_diff
                        offset[idx_] = onset[idx_+1] - mean_tt_diff
                    
                    response_time = 0 ### time from directional cue until end of trials
                    # duration = [response_time] * len(onset)
                    duration =  np.subtract(offset, onset) # [event_duration] * len(onset)
                    
                    #
                    # get trial_types and replace 
                    trial_type = [{'f':'Face', 'o':'Other'}[el[9:10]] for el in trl_list[0].tolist()]
                        
                    
                #
                # go
                elif 'go' in run_:
                    print(run_)

                    # trl_list = pd.read_csv( opj( info.data_paths.bids_dir, sub_id, ses_, 'nrec/trial_lists', info.func_run_dict[sub_id][ses_][iis]+'.txt') , sep='\t', header=None)
                    trl_list = pd.read_csv( opj( info.data_paths.bids_dir, sub_id, ses_, 'nrec/trial_lists', run_+'.txt') , sep='\t', header=None)
                    gaze_targets = [tt[1:2] for tt in trl_list[0].tolist()]
                    nt_trials = [int('_nt' in el) for el in trl_list[0].tolist()]
                    mat = loadmat( opj( info.data_paths.bids_dir, sub_id, ses_, 'nrec', mat_files_of_runs[iis] ) )

                    onset = mat['TrialList'][:,8].tolist()
                    offset = mat['TrialList'][:,9].tolist()

                    # repair missing trial times
                    tt_diff = np.abs(np.subtract(onset[1:], offset[:-1]))
                    mean_tt_diff = np.mean(tt_diff[np.intersect1d(np.where(tt_diff > 0.2), np.where(tt_diff < 1))])
                    idxs = np.add(np.where(np.array(onset[1:]) == 0.0)[0], 1)
                    for ii, idx_ in enumerate(idxs):
                        onset[idx_] = offset[idx_-1] + mean_tt_diff
                        # offset[ii+idx_] = onset[ii+idx_]+30
                    
                    if onset_alignment == 'cue':
                        ### add ITI from trial_list to trial onsets from nrec to get the onsets of the spatial cue
                        if sub_id in ['sub-00', 'sub-000']:
                            onset = np.add(onset, trl_list[4].tolist()).tolist()
                        else:
                            onset = np.add(onset, trl_list[5].tolist()).tolist()
            
                    response_time = 0 ### time from directional cue until end of trials
                    # duration = [response_time] * len(onset)
                    duration = [event_duration] * len(onset)

                    #
                    # get trial_types and replace 
                    trial_type = trl_list[1].tolist()
                    trial_type = ['Gaze_'+str(gaze_targets[ii]) for ii,x in enumerate(trial_type)]  
                    nt_trials = np.where(['_nt' in el for el in trl_list[0].tolist()])[0]
                    for idx in nt_trials:
                        trial_type[idx] = 'Ignore'

                #
                # co
                elif 'co' in run_:

                    print(run_)

                    # trl_list = pd.read_csv( opj( info.data_paths.bids_dir, sub_id, ses_, 'nrec/trial_lists', info.func_run_dict[sub_id][ses_][iis]+'.txt') , sep='\t', header=None)
                    trl_list = pd.read_csv( opj( info.data_paths.bids_dir, sub_id, ses_, 'nrec/trial_lists', run_+'.txt') , sep='\t', header=None)
                    if sub_id in ['sub-00', 'sub-000']:
                        cube_targets = trl_list[3].tolist()
                    else:
                        cube_targets = trl_list[4].tolist()
                    nt_trials = [int('_nt' in el) for el in trl_list[0].tolist()]
                    mat = loadmat( opj( info.data_paths.bids_dir, sub_id, ses_, 'nrec', mat_files_of_runs[iis] ) )

                    onset = mat['TrialList'][:,8].tolist()
                    offset = mat['TrialList'][:,9].tolist()
                    
                    # repair missing trial times
                    tt_diff = np.abs(np.subtract(onset[1:], offset[:-1]))
                    mean_tt_diff = np.mean(tt_diff[np.intersect1d(np.where(tt_diff > 0.2), np.where(tt_diff < 1))])
                    idxs = np.add(np.where(np.array(onset[1:]) == 0.0)[0], 1)
                    for ii, idx_ in enumerate(idxs):
                        onset[idx_] = offset[idx_-1] + mean_tt_diff
                        # offset[ii+idx_] = onset[ii+idx_]+30

                    if onset_alignment == 'cue':
                        ### add ITI from trial_list to trial onsets from nrec to get the onsets of the spatial cue
                        if sub_id in ['sub-00', 'sub-000']:
                            onset = np.add(onset, trl_list[4].tolist()).tolist()
                        else:
                            onset = np.add(onset, trl_list[5].tolist()).tolist()
            
                    response_time = 0 ### time from directional cue until end of trials
                    # duration = [response_time] * len(onset)
                    duration = [event_duration] * len(onset)

                    #
                    # get trial_types and replace 
                    trial_type = trl_list[1].tolist()
                    trial_type = ['Cubes_'+str(cube_targets[ii]) for ii,x in enumerate(trial_type)]  
                    nt_trials = np.where(['_nt' in el for el in trl_list[0].tolist()])[0]
                    for idx in nt_trials:
                        trial_type[idx] = 'Ignore'

                #
                # gic
                elif run_.split('_')[0] in ['eic', 'gic']:

                    print(run_)

                    # trl_list = pd.read_csv( opj( info.data_paths.bids_dir, sub_id, ses_, 'nrec/trial_lists', info.func_run_dict[sub_id][ses_][run_.split('_')[0]][gic_counter]+'.txt' ) , sep='\t', header=None)
                    trl_list = pd.read_csv( opj( info.data_paths.bids_dir, sub_id, ses_, 'nrec/trial_lists', run_+'.txt' ) , sep='\t', header=None)
                    eye_targets = [tt[1:2] for tt in trl_list[0].tolist()]#trl_list[2].tolist()
                    if sub_id in ['sub-00', 'sub-000']:
                        cube_targets = trl_list[3].tolist()
                    else:
                        cube_targets = trl_list[4].tolist()
                    iris_targets = [iris_color_tTD_mapping[tt[2:4]] for tt in trl_list[0].tolist()]

                    mat = loadmat( opj( info.data_paths.bids_dir, sub_id, ses_, 'nrec', mat_files_of_runs[iis] ) )                    
                    
                    onset = mat['TrialList'][:,8].tolist()
                    offsets = mat['TrialList'][:,9].tolist()

                    # repair missing trial times
                    tt_diff = np.abs(np.subtract(onset[1:], offsets[:-1]))
                    mean_tt_diff = np.mean(tt_diff[np.intersect1d(np.where(tt_diff > 0.2), np.where(tt_diff < 1))])
                    idxs = np.add(np.where(np.array(onset[1:]) == 0.0)[0], 1)
                    for ii, idx_ in enumerate(idxs):
                        onset[idx_] = offsets[idx_-1] + mean_tt_diff
                        # offsets[ii+idx_] = onsets[ii+idx_]+30

                    if onset_alignment == 'cue':
                        
                        #
                        if sub_id in ['sub-00', 'sub-000']:
                            onset = np.add(onset, trl_list[4].tolist()).tolist()
                        else:
                            onset = np.add(onset, trl_list[5].tolist()).tolist()
                        response_time = 0  # time from directional cue until end of trials (response time = 2s), set to 0 to model event as a delta pulse
                        # duration = [response_time] * len(onset)
                        duration = [event_duration] * len(onset)

                    elif onset_alignment == 'instruction':
                        # duration = trl_list[4].tolist()
                        duration = [event_duration] * len(onset)

                    if onset_offset:
                        onset = np.add(onset, onset_offset)

                    ### get trial_types and replace 
                    trial_type = trl_list[1].tolist()

                    for ii, x in enumerate(trial_type):

                        if x=='Wuerfel.mp3':
                            trial_type[ii] = 'Cubes_'+str(cube_targets[ii])

                        if x=='Augen.mp3':
                            trial_type[ii] = 'Gaze_'+str(eye_targets[ii])

                        if x=='Farbe.mp3':
                            trial_type[ii] = 'Iris_'+str(iris_targets[ii]) 

                #
                # gaze, cubes (sub-00)
                if 'gc' in run_ and run_.split('_')[0] in tasks:

                    print(run_)
                    
                    nrec_dir = np.sort(os.listdir('/media/marius/data_ex/gaze-motion/dsgm/sub-00/nrec')) #np.sort(os.listdir(opj( info.data_paths.bids_dir, sub_id, ses_, 'nrec' )))
                    mat_files_of_runs = [ff for ff in nrec_dir if '.mat' in ff and not 'calibration' in ff]

                    ### read run_-x.txt and nrec file
                    trial_list_dir = os.listdir('/media/marius/data_ex/gaze-motion/dsgm/sub-00/nrec/trial_lists')
                    
                    trl_list = pd.read_csv( opj( '/media/marius/data_ex/gaze-motion/dsgm/sub-00/nrec/trial_lists', 
                                                [sess for sess in trial_list_dir if run_[-1] in sess][0] ) , sep='\t', header=None)
                    eye_targets = trl_list[2].tolist()
                    cube_targets = trl_list[3].tolist()
                    mat = loadmat( opj( info.data_paths.bids_dir, sub_id, 'nrec', mat_files_of_runs[iis]) )

                    #
                    # SPATIAL CUE onsets + RESPONSE durations
                    onset = mat['TrialList'][:,8].tolist()
                    offset = mat['TrialList'][:,9].tolist()

                    if onset_alignment == 'cue':

                        onset = np.add(onset, trl_list[4].tolist()).tolist()
                        onset = np.add(onset, 0).tolist() ## add an offset to the onsets, eg. to capture later components of the response
                        response_time = 0  # time from directional cue until end of trials (response time = 2s), set to 0 to model event as a delta pulse
                        # duration = [response_time] * len(onset)
                        duration = [event_duration] * len(onset)

                    elif onset_alignment == 'instruction':

                        duration = [event_duration] * len(onset)

                    if onset_offset:
                        onset = np.add(onset, onset_offset)

                    # WIT DIFFERENTAITING VERTICAL OR HORIZONTAL CUBE ALIGNMENT 
                    # get trial_types and replace 
                    # trial_type = trl_list[1].tolist()
                    # if sub_id in ['sub-00', 'sub-000']:
                    #     current_cube_arrangement = run_identifier[iis][-1]#session_sequence.pop(0)
                    #     trial_type = ['Cubes_'+str(cube_targets[ii])+'_'+current_cube_arrangement if x=='Wuerfel.mp3' else 'Eyes_'+str(eye_targets[ii]) for ii,x in enumerate(trial_type)]

                    # WITHOUT DIFFERENTAITING VERTICAL OR HORIZONTAL CUBE ALIGNMENT 
                    # get trial_types and replace 
                    trial_type = trl_list[1].tolist()
                    trial_type = ['Cubes_'+str(cube_targets[ii]) if x=='Wuerfel.mp3' else 'Gaze_'+str(eye_targets[ii]) for ii,x in enumerate(trial_type)]


                #
                # make dict
                events_dict = { 'onset':[round(x, 2) for x in onset], 'duration':[round(x, 2) for x in duration], 'trial_type':trial_type }

                # remove trials with missing trial times
                if 'gic' in run_:
                    idx_to_rm = idx_to_rm = [ii for ii, el in enumerate(np.subtract(onset , trl_list[5].tolist())) if el < ii*3.5]
                    for kk in events_dict:
                        events_dict[kk] = list(np.delete(events_dict[kk], idx_to_rm) )

                #
                # make Dataframe
                # evd.append(events_dict)

                #
                # save to ROOT/SUB/func folder
                # pd.DataFrame(events_dict).to_csv( opj( info.data_paths.bids_dir, sub_id, ses_, 'func', sub_id+'_'+ses_+'_task-'+run_+'_events.tsv' ), sep='\t', index=None)

                #
                # save to FMRIPREP_ROOT/SUB/func
                pd.DataFrame(events_dict).to_csv( opj( info.data_paths.fmriprep_dir, sub_id, ses_, 'func', sub_id+'_'+ses_+'_task-'+run_+'_events.tsv' ), sep='\t', index=None)


cue
*** Processing subject:  sub-00
gc_run-1
gc_run-2
gc_run-3
gc_run-4
gc_run-5
gc_run-6
